In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/Nikhil-V98/Analyticsvidhya/main/Black_friday/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Nikhil-V98/Analyticsvidhya/main/Black_friday/test.csv')

In [3]:
test.fillna(0,inplace=True)
train.fillna(0,inplace=True)

In [5]:
col_name = train.columns

In [6]:
gender_dict = {'F':0, 'M':1}
age_dict = {'0-17':17, '18-25':25, '26-35':35, '36-45':45, '46-50':50, '51-55':55, '55+':60}
city_dict = {'A':0, 'B':1, 'C':2}
stay_dict = {'0':0, '1':1, '2':2, '3':3, '4+':4}

train['Gender'] = train['Gender'].map(gender_dict)
test['Gender'] = test['Gender'].map(gender_dict)

train['Age'] = train['Age'].map(age_dict)
test['Age'] = test['Age'].map(age_dict)

train['City_Category'] = train['City_Category'].map(city_dict)
test['City_Category'] = test['City_Category'].map(city_dict)

train['Stay_In_Current_City_Years'] = train['Stay_In_Current_City_Years'].map(stay_dict)
test['Stay_In_Current_City_Years'] = test['Stay_In_Current_City_Years'].map(stay_dict)


In [7]:
testRes = test[['User_ID']]
testRes['Product_ID'] = test['Product_ID']

C:\Users\Nik\AppData\Local\Temp\ipykernel_10824\3452910973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testRes['Product_ID'] = test['Product_ID']


In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
scaler = StandardScaler()

train['User_ID']=enc.fit_transform(train['User_ID'],)
test['User_ID']=enc.transform(test['User_ID'])

In [9]:
# Standard scalar
train['Product_ID'] = train['Product_ID'].str.replace('P00', '')
test['Product_ID'] = test['Product_ID'].str.replace('P00', '')

train['Product_ID'] = scaler.fit_transform(train['Product_ID'].values.reshape(-1, 1))
test['Product_ID'] = scaler.transform(test['Product_ID'].values.reshape(-1, 1))

# Label Encoding Product_IDs
new_product_ids = list(set(pd.unique(test_data['product_id'])) - set(pd.unique(train_data['product_id'])))

le = LabelEncoder()
train_data['product_id'] = le.fit_transform(train_data['product_id'])
test_data.loc[test_data['product_id'].isin(new_product_ids), 'product_id'] = -1
new_product_ids.append(-1)

test_data.loc[~test_data['product_id'].isin(new_product_ids), 'product_id'] = le.transform(test_data.loc[~test_data['product_id'].isin(new_product_ids), 'product_id'])

# Drop User ID and Product ID from test and train


In [10]:
categorical_col = ['Gender', 'City_Category','Marital_Status']
numerical_col = ['Age', 'Occupation', 'Stay_In_Current_City_Years', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']

In [11]:
for col in categorical_col:
    train[col] = enc.fit_transform(train[col])
    test[col] = enc.transform(test[col])

In [12]:
for col in numerical_col:
    train[col] = scaler.fit_transform(train[col].values.reshape(-1, 1))
    test[col] = scaler.transform(test[col].values.reshape(-1, 1))

In [13]:
from sklearn.model_selection import train_test_split
X = train.drop(['Purchase'], axis=1)
y = train[['Purchase']]
X_test = test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

In [14]:
xgb_reg_1 = XGBRegressor(n_estimators=400, max_depth = 20, learning_rate = 0.05,  min_child_weight  = 40)

xgb_reg_1.fit(X_train, y_train)

y_pred = xgb_reg_1.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_pred, y_val))

print (xgb_reg_1)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=20, max_leaves=0, min_child_weight=40,
             missing=nan, monotone_constraints='()', n_estimators=400, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)


NameError: name 'model_performance' is not defined

In [15]:
print(f'Root Mean Squared Error of the model is : {rmse}')

Root Mean Squared Error of the model is : 2509.3071749707287


In [17]:
yPreds = xgb_reg_1.predict(X_test)
testRes['Purchase'] = yPreds
submission = testRes[['User_ID', 'Product_ID', 'Purchase']]

submission.columns = ['User_ID', 'Product_ID', 'Purchase']
submission.to_csv('submission_V1.csv', index = False)
submission.head()

C:\Users\Nik\AppData\Local\Temp\ipykernel_10824\558239347.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testRes['Purchase'] = yPreds


,User_ID,Product_ID,Purchase
0,1000004,P00128942,14954.965820
1,1000009,P00113442,11484.133789
2,1000010,P00288442,7125.341797
3,1000010,P00145342,2267.904053
4,1000011,P00053842,2239.421631
